# 1.1: Downloading and preparing `librispeech` files

우리는 `project gutenberg`에서 제공하는 무료 데이터셋인 `librispeech` 파일을 사용할 것입니다. `librispeech`는 원어민 화자들이 책을 읽는 것을 녹음한 데이터입니다. 

언어 모델과 Lexicons에 대한 설명은 [이 곳](http://openslr.org/12/)에서 확인하실 수 있습니다. 

**Note:** `librispeech`를 직접 다운로드 할 필요는 없습니다. 아래의 스크립트를 실행하시면 자동으로 필요한 파일들을 다운받으실 수 있습니다. 

## Downloading the audio

In [1]:
# location to download raw audio
data=${KALDI_INSTRUCTIONAL_PATH}/raw_data
mkdir $data

# base url for downloads
data_url=www.openslr.org/resources/12
lm_url=www.openslr.org/resources/11

# source files with path information
. ${KALDI_INSTRUCTIONAL_PATH}/path.sh

mkdir: cannot create directory '/scratch/kaldi/egs/INSTRUCTIONAL/raw_data': File exists


오디오 파일의 세부 내용은 [이 곳](http://www.openslr.org/12/)에서 확인하실 수 있습니다. 

오디오 파일은 크게 두 세트, `clean`과 `other`로 나눌 수 있습니다. `clean`은 아주 정확하게 발음되어서 `ASR`에서 인식하기 쉬운 편에 속하는 오디오 파일을 모아놓은 데이터셋이며, `other`은 상대적으로 `ASR`에서 인식하기 어려운 데이터를 모아 놓은 데이터셋입니다. 또한 녹음된 양에 따라서 `100 hrs`, `360 hrs`, 그리고 `500 hrs`의 세 데이터셋이 존재합니다. 

이번 워크샵에서는 가장 짧은 데이터셋인 `train-clean-100` 데이터셋으로 모델을 훈련시키고, `test-clean`과 `test-other` 데이터셋을 이용하여 `ASR` 모델의 성능을 확인할 것입니다.

아래 셀의 명령어를 실행하시면 `INSTRUCTIONAL/raw_data` 아래에 다음과 같은 데이터셋을 다운로드합니다. 

 - `train-clean-100`
 - `dev-clean`
 - `dev-other`
 - `test-clean`
 - `test-other`

**Note:** 이 과정은 인터넷 속도에 따라 한 시간 이상의 시간이 소요될 수 있습니다. 

In [2]:
for part in dev-clean test-clean dev-other test-other train-clean-100; do
    ${KALDI_INSTRUCTIONAL_PATH}/local/download_and_untar.sh ${data} ${data_url} ${part}
done

/scratch/kaldi/egs/INSTRUCTIONAL/local/download_and_untar.sh: downloading data from www.openslr.org/resources/12/dev-clean.tar.gz.  This may take some time, please be patient.
--2018-06-16 05:30:59--  http://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: 'dev-clean.tar.gz'

dev-clean.tar.gz    100%[===================>] 322.27M   381KB/s    in 7m 24s  

2018-06-16 05:38:24 (744 KB/s) - 'dev-clean.tar.gz' saved [337926286/337926286]

/scratch/kaldi/egs/INSTRUCTIONAL/local/download_and_untar.sh: Successfully downloaded and un-tarred /scratch/kaldi/egs/INSTRUCTIONAL/raw_data/dev-clean.tar.gz
/scratch/kaldi/egs/INSTRUCTIONAL/local/download_and_untar.sh: downloading data from www.openslr.org/resources/12/test-clean.tar.gz.  This may take some time, pl

## Converting the audio

`ffmpeg`라는 명령어를 이용해서 `librispeech` 내부의 오디오 파일들을 `16kHz`에서 `8kHz`로 `downsampling`을 하고, `flac` 파일을 `wav` 파일로 변환하겠습니다. (또한 `16-bit signed little endian encoding`으로 변환됩니다.)

또한 나누어져있는 `train`, `dev`, 그리고 `test` 데이터셋을 하나의 디렉토리에 모아두도록 하겠습니다.  directories:
 - `train_clean_audio`
 - `dev_clean_audio`
 - `dev_other_audio`
 - `test_clean_audio`
 - `test_other_audio`
 
**Note:** 이 과정은 `한 시간` 이상 소요될 수 있습니다. 

In [ ]:
for part in dev-clean test-clean dev-other test-other train-clean-100; do
    ${KALDI_INSTRUCTIONAL_PATH}/utils/data/convert_audio_directory.sh \
        -i ${KALDI_INSTRUCTIONAL_PATH}/raw_data/LibriSpeech/${part} \
        -o ${KALDI_INSTRUCTIONAL_PATH}/raw_data/LibriSpeech/${part}_audio \
        -s 8000
done

그리고 다음 명령어를 통해서 파일 이름들을 정리하도록 하겠습니다. 

In [ ]:
for part in dev-clean_audio test-clean_audio dev-other_audio test-other_audio train-clean-100_audio; do
    ${KALDI_INSTRUCTIONAL_PATH}/utils/data/strip_duplicate_filetype.sh \
        ${KALDI_INSTRUCTIONAL_PATH}/raw_data/LibriSpeech/${part}
done

## Downloading `language model`s and `lexicon`

`language model`과 `lexicon`에 관련된 파일들도 생성되어 있습니다. 

`librispeech-lm-norm.txt.gz`는 `언어 모델 (language models)` 생성에 필요한 `text`를 압축해놓은 파일입니다. <br>
`librispeech-lexicon.txt`는 `ASR`에 사용되는 모든 단어와 그 발음들이 저장된 파일입니다. <br>
`3-gram.arpa.gz`는 `3-gram` `language model`을 압축해서 저장한 파일입니다.. <br>
`3-gram.pruned.1e-7.arpa.gz`와 `3gram.pruned.3e-7.arpa.gz`은 각각 `n-gram`의 확률을 기준으로 `3-gram.arpa.gz`을 `pruning`한 파일입니다. <br>
`4-gram.arpa.gz`는 `4-gram` `language model`을 담고 있습니다. <br>

다음 셀의 명령어는 위의 파일들을 `INSTRUCTIONAL/raw_data` 디렉토리에 다운로드합니다. 

**Note:** 이 과정은 `한 시간` 이상 소요될 수 있습니다. 

In [ ]:
${KALDI_INSTRUCTIONAL_PATH}/local/download_lm.sh ${lm_url} ${data}

다음 과정을 간단하게 하기 위해서, `언어 모델 (language models)`에 해당하는 파일들은 압축을 풀도록 하겠습니다. 

In [ ]:
for lm in 3-gram.arpa.gz 3-gram.pruned.1e-7.arpa.gz 3-gram.pruned.3e-7.arpa.gz 4-gram.arpa.gz; do
    gzip -df raw_data/${lm}
    echo "uncompressed ${lm}"
done

이전 과정에서 생성된 `symbolic links`들은 더 이상 필요가 없으므로 제거하도록 하겠습니다. 

In [ ]:
rm raw_data/lm_*.gz

In [ ]:
ls raw_data

## Fixing bugs in `lexicon`

다운받은 `lexicon`에는 `SPIRITS`에 해당하는 동일한 발음이 두 개가 존재합니다. 

In [ ]:
cat raw_data/librispeech-lexicon.txt | grep SPIRITS

그러므로, 우리는 두 번째 발음을 다른 발음으로 교체하도록 하겠습니다. (자세한 내용은 나중에 다루겠습니다.)

```
SPIRITS  S P IH1 R IH1 T S
```

In [ ]:
cat raw_data/librispeech-lexicon.txt | \
    perl -pe 's{S P IH1 R IH0 T S}{++$n == 4 ? "S P IH1 R IH1 T S" : $&}ge' \
    > raw_data/librispeech-lexicon.txt.corrected
    
mv raw_data/librispeech-lexicon.txt.corrected raw_data/librispeech-lexicon.txt

In [ ]:
cat raw_data/librispeech-lexicon.txt | grep SPIRITS

`kaldi`는 또한 `lexicon`에 `<unk>`가 포함되어 있음을 가정합니다. 그러므로 `lexicon`에 `<unk>`를 추가하겠습니다. `<unk>`는 `lexicon`에 등록되어 있지 않은 단어를 `kaldi`에서 인식하였을 경우를 처리하기 위한 방법입니다. 


```
<unk>    SIL
```

In [ ]:
printf "<unk>\tSIL\n" >> raw_data/librispeech-lexicon.txt

In [ ]:
tail -n3 raw_data/librispeech-lexicon.txt

## Building a transcript file

다음 명령어는 많은 파일들을 자동으로 생성합니다. 해당 파일들은 나중에 기회가 되면 자세하게 다루도록 하겠습니다. 

In [ ]:
for part in dev-clean test-clean dev-other test-other train-clean-100; do
    data_files=${data}/LibriSpeech/${part}_data
    mkdir -p ${data_files}
    ${KALDI_INSTRUCTIONAL_PATH}/local/data_prep.sh \
        ${data}/LibriSpeech/${part} \
        ${data_files}
    rm -r ${data}/LibriSpeech/${part}    # we no longer need the original audio, so we can delete it
done

위에서 생성한 파일들을 바탕으로 음성 파일들의 전사를 담고 있는 하나의 파일을 만들 수 있습니다. 

In [ ]:
for part in dev-clean test-clean dev-other test-other train-clean-100; do
    data_files=${data}/LibriSpeech/${part}_data
    cat ${data_files}/text
done > ${data}/librispeech-transcripts.txt

In [ ]:
head -n5 raw_data/librispeech-transcripts.txt

## Building a phones file

다음 명령어는 `lexicon`에서 사용된 모든 `phone`을 담은 파일을 생성합니다. 

In [ ]:
python local/build_phones_list.py \
    ${data}/librispeech-lexicon.txt \
    ${data}/librispeech-phones.txt

In [ ]:
head -n5 raw_data/librispeech-phones.txt